นางสาวพรธวัล คุ้มภัย INE 6506022610031

In [ ]:
!pip -q install pandas scikit-learn openpyxl streamlit pyngrok joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.0 MB/s eta 0:00:00


In [84]:
import os, re, json, random
import pandas as pd, joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [127]:
FACTS = {
    "year": "2025",
    "team_short": "F1 Team Red Bull",
    "team_full": "Oracle Red Bull Racing",
    "drivers": ["Max Verstappen", "Liam Lawson"],
    "team_principal": "Laurent Mekies",
    "technical_director": "Pierre Waché",
    "car_name": "Red Bull Racing RB21",
    "power_unit": "1.6L V6 turbo-hybrid (Honda RBPTH003)",
    "base_chassis": "Milton Keynes, UK",
    "base_engine": "Milton Keynes, UK",
    "constructors_points_asof": 239,
    "asof_event": "Italian Grand Prix",
    "asof_date": "2025-09-10",
    "max_song" : "https://youtu.be/mS41bHNw1WI?si=idnXKPox1kYTav-e",
    "max_picture" : "data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMTEhUSEhMWFhUWGB0aGBcYGRoXFxogHR4eGBgeHhoaHSggGh0lGxgXITEhJSorLi4uFx8zODMtNygtLisBCgoKDg0OGxAQGy0lHyUtLSsvLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0rLS0tLS0tLS0tLS0tLS0tLS0tL//AABEIALcBEwMBIgACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAAFAAMEBgcCAQj/xABIEAACAQIEAwUEBwUHAgQHAAABAgMAEQQSITEFBkETIlFhcTKBkaEHFCNCUrHBFZLR4fAWM1NicqLxQ1RjgsLSJkR0g7LT4v/EABkBAAMBAQEAAAAAAAAAAAAAAAABAgMEBf/EACwRAAICAgICAQIDCQAAAAAAAAABAhESIQMxE0EiUWFxgdEEFDIzkaHB8PH/2gAMAwEAAhEDEQA/ANMw2ISQXRgw8jen8tYtyrzkMMDAlrXJzbaZr2HibaeVabwrnDCTIp7QKxt3DuL6VceRMifDKJO45BO0LLhnCS6WY6jz+VULlbCYiPjBXFSCSTsC2Ya6Eiw8tjWnwsrC6kEeI1qk4aP/AOIJP/pV/wDypv0RH2i6EV5aucXiY4xd3C9NTTtqqyaG7V7XWWllosKPBXte2r0LRYUICu012N7aaV6or5exPH8fhsTPmaSGV5A8qEBLsrCRGZVUKWuASwHeBPRqTlRSjZv3My3nw67AiS5vawtvUf6OuHwxQOsGIM4zm5LZrHqK5w/Ekx5wM6Zcs0bllvfKbAOvqrXHuo5y5y7Dg4zHCuUM2Y+tABArXlqiYTiqSTSQqGzR2zXUga+B2PuqfloFQ3lr3LTXEcYkEbSyEKq7k6VWuNcfM0bLhCC4GoB7ytva3jrSckuyo8cpdItWWlaq9yjzKuJURvpMB3lO4tobjoasuWmnZNDYFdBa7C1WOdudYeHhUNpMRJ/dxZgtv8zndE91ydAN7FhRTuf/AKUDh5p8HAMrIUTtwAxU2LS2UmxYXRVBsL5ydAKtn0d8yfX8GJrWZGMbAsHa6gd5iEUZmBB0FtfcMj5a5MxPF8bPLiGWNBLmnddyX72WMa/dtqfZBF7nQ79w3hcWHiWGCNY40FlVf1O5J6k6nrUpsqSXR0RXhp0rXBFXZFDdeU5avCKLChs14actXlqLChuvKctXlqLChuvDTtq8tSsKGrUqdsKVFjo+XYRYtlAYNoLefhRDE4wRoyqCsl8rEm+nW1C5FeJLi4UnQ/zqVkR1JL982NvHx18axaTOuHI46NH5c5hOBhSQozmVbqmbujzPh08ajjm134hNiYUAdoFjAPRr/wATVcx/EHVizsFAjVQAN/SmYnUF2ViNEsTve/lUZPo0cIOpLu9o0fhXKeKllhnxLZirB2u1/QADQVpJWsZ5b5tOGWRgzysWtZiSB7jrVrwH0gXIMq5RqTbW46WquNxiqMudTnutfYvOWllqJwDiQxMXaqLKSbX8KjcT5nw0EixSSAM2wrbJVZzUFQtehaGT8yYZRcyLvbfxrvFcwQRukbN3ntYDzpZIKCZIAJOwqt8x8t4LisI7QDOAeylU2dff95b/AHTce/WiHG+MJDYP7L6X9azTFy4nBMxzWja/Zm+gB1tSbGkUTDcSn4ZKqxK0csEpEztcxuRpa3VGAFmABsw6m9bRy59JGGxiZGPYzlfZYgqx/wAj9fQ26jWseusrN2pzhgb6m+vgaBYnAtCuYMWT4Mt9N+h26WN6hTNUovTPpfgIl7aXPIjLZbKB3hprejuWsB5E5+bCszSASrIoBa1pAVGmv4dTdTtc2030nl7n6Bo7zObNmKuR3Tqe6COo291XkmRPjlF7K59IHMokmjgdHCRvuNVL6Fbkbb9el/GhnBeOx4YlmVWFvZUXYm4tYBzpoBbbu71W+Y8WJ55pUYAXPdJsxF+6+UnXTu9LAddaiczYKRisnZPEgAVrgDvWuBYdbUvj22bQXJSUV31+Jp2Hx9uJR4gwND9ZAzqSpOa1rnLcC9gbe/rWnBKxXl1lwywRiRXUsGeW4yBmbRAxsT3QDcge1sNqm/SB9KWa+F4a+wtNiANhsVjzdf8AP06eIH8W/oZ4OUqXZN+kT6Thhy2FwLK0oB7SfRkh6WA2eTy2BsNTcDNeTOW5+L4pixZo1IaeaRrsb6asblnIBsuwsNhvE5X5dm4hiVigiYQhgJJAt0jB0DNcgGw2F7m3nW+rxzh/DYlwquF7JQLaFmIGpaw1Y7k1N3tlSqHxj/X/AH0MQPg+DxDDxXIeRnsWDOWbXU7mwAUdbKN965xvPsCILg5zsvW3jVJ5l4ynFJkXDRgMoPfOhNVfmSQPLHHZlaNcrX8RUS5GjOjdOWeJnEw9oQLE6elFGWqhybiPq+EVChBte17k+dXKCQOoNaxlaJaGiteFKh8a4okQRcwDSNlTXrVFwPM80c7fWJhaNypTTXqNeulqrIVGiFa8y13hZM6Bx1F6WIkVFLsbKouTTsKOMtLJQg804Zo5GjkUlFv8qpv9v8UREexVVLd6S91IHtADobVLmh4s0jJXmSh7cyYfsmlzjugEjrrqNPOgPGPpHwsKRut3zi9huvrfbWnkhUW7JXlZiPpaP/bj974dKVR5Ihiyk8vYorawEgF7q2o1Fjag3GsOvaF4lKJp3PDxt5UWwEzIyKCAwBzC2o6WrzHyh4rDVh06766VnH4rs75pTl0BRjw4tICwVTYE21O3rUzDNlgZrXN039aGTQ3GmhogpK4Vr6kMn501JSMp8codk3h+LZpVkYqpZ7EDa9utE8BiZ5pG7JcwDFDa2VQD4+f60BhmEzBe6LG/lT0fFDAojiYr37uQbZh4VVLsFN6Rf+HcyYqBDEir3thb2ap3F2lYiSYEkHfxohgcd2spZDl7txcb9LflU/iOIQxZWHeFr66f1tWd3Vmj41K2gBDjiwy3st71Jx2LAdHLk6b32pScFKgFB3jsviN6AYgm9n01PuqMbdmMouOmFOYeY5Jyis7FF2BNR8fx+WaII5LBdvQbXoViSBsb00kxFbLozC2BxNwTYXp9kz5gLEW1vULh6DJIRvapvDsOzLva+96h9jBXEeF9i2aAlltqvX3eP5+tC8Q47rxkj00ynyttVxigjz2YknraoHFeHxFHaMWb1310uNr36+tGVOzSMrWD/L7AHCYiSNllVTcahmBZWYG5vfe+gI9KN4nm2eVHUJGmcEsb3GmncB0BFh4nQetDsDDG8YU4lkJOqalSS1h3bWGmVr3PXQWpyDCoGzieKxNguZTa975ryKdO6brfw2GurRCkwficQxRVLXAv1v6H33O/h5UV5W4BNjpUwsAClrGRzqqKd2PmRbujfQeNRsbAkgTJIrOB9qFUKq3tbKVXvddSTt0q88tI0AXsmKFwC5X2tdT8NvdSlVpMqMnFNr3r9TauVeWoMBhxh8Otl3djqztaxZj46DTYdKy36RxE8jx4bClpFJ7STL79+tWrBcZkhgd2kdgfZzD8jbWqpx76Q2KSRog7y2z21BptoxM7wkjxkSLdTfcaU9JKzS3ZixJvc701LjHkADHRTppXjSa6GxtvWMkMtfDOZpsM/aOS7DQDcWqFjOdcSXd0kZMxvlB0HxoLhZyXAO3WmscwZjr1qVd0wCuAx3ayBpnc72YsSVPQi5qfwFY3lY4h+6puT95jtvVVhQk21tTsmILaLvV7QzceR+I4eGByZiyltMxJIGwAB29KFfSJz1H2T4aInM2ha2luvvqk4DjQwsNuzBkZh3qCcRzztnfcn0p56oKG4MQfuk67+Yon27LEqpLpe5Q7C+9qiR4Jkt4Wpz6wL3yjTS9ZZU9DCXbqyGVdDHHlYHUNvrb30F5n4UYJEAbOrxrIDse9uLeRvUjjHDJIlWXTI9tQdPGxqBieIO7ZpDm0sL7AeXhWq6ERw196VPCRfClSAufGeU5MHDEZcrM0ljIDqL6++h/E8MyyRlUz6tdl6jYe+1Wj6XsH2axPnYq0o+zv3RYdKsGN5ThkjjKKI3JBup02633qsTbyMy7iHCPs2cgKAbj3nY1DwWE7UNCDuyjy+NaTxfhKrhpdMsiOEY3JQ3Aa9j5EVn/CJMmdlGa5AH9dalqi1Nyr8SHJypiY5bKtrkrm3XQX39KG47DmI3kIJJsbDQVdU4kCgZncEOTkG2uhrriRjeEzDKxzhOzYd4A/e/rwpqTsHCL2VQzNGxK3AKiwPSpPLfFynamRO0Q6nN40Wx+EVmXKrGy7Wtt+lOYjgIYSIkmUvlNiLAeNzQ3svjTim0ziPjMbqsxfIwawA6Dwo00EWIwy3iAYknMd2v1qlYrlp1zqDmK7dAatHJmKZ0OGlGVgO4za/CnVE53pqircU5aliTOLMuu24oCa2BeXZXLDPcFbAeJoLxDkuJorAFZhv4e+nddmTgqtFLwjWikt5U9gsWbBT1O9PcU4QcMroXDXAIIqNwsDTNvfSh0ZtVph4QCOYFjra9B8diwWZgNCTRLjM5+sFTqMmlVrtDtSSsLoe4aY0GWWMtmltGcoIB9nU5vlbpf0s3EeOYSTCRg2YDXL2RzADRegVLai4OpHXWq1h8S95FURkICe+bHvXYkfiPSw1103JqKvFNSoRbEBQLgL9/cZRp9p5bak61z8nB5GpO9b7PShzQh8bVfeN7/yS4nV5EaJCsZXJYgaEZm3zHN1102qzLjSCSpJsflVXw8hLhbxgAMQqNmOiN1GlrXotw+XulTWzTpWcXNi38f0+n9g/wAR53d8L9Vyiw+91tVMnkPXrT+JdQ1lFR5XvYW2quzA57XS1cgmxriTyrhp9KdAPRyda9SYCoiC9PItDSAn4SS5t46U/wBmgdh7JFDI5iNt6U05Y3O5qHHYE15SzZF36U+c4Yq+9QcBLlZSf50cGDkMpUjXVlvtb+jUuL9DIk2KIXL1FRZJ+4DU7hfAmmlszWVrm430qdxPlgRTnD9oSAua9UuOhlfkxblbFjl3y9L+lRUJLWo7DwpCGDE9w2HwvXqYSMQF8ozFTrWmIgSIfOlRv9pRIArDUAX08qVTQ6ND+kyMPFDm1Ge4v00q1YdnyRXbw/Ksw49zlDiYIDqLGxO5vbYje9XDgXNkM8iYYB1lUah0IuAN7++qGNceLSw4lGOnbqNND7K1VcLwZAGCkrZhYkjTw3qz8TY9nibdcR/6RVO45iWjhZirMuYZstr+R1/SnFJumJtpaJmC5dZUkzMSGJKvofMip64BSfZHdsSdr0z9H8zYu8bMYoyt0zgEvJex1Gy2Fh460TxMbxvLG4sy6GnOCjLQ4zlQJ5jh+zZxnU3Auvgf0qO3DnEBYTLIJTpbdbaW9aMT40hHB2v+lVDD8zx/V2jIyiNiVH3jrrWMryVFrkktIJ4rhjSIqh8pQDNprXWAwJBBuGA0J2NdcK4isytMmzLqPAjQipXDG7hIq6Fmwc/EJo3UR9pe+vXSp2H4pKRZg+psWI1t1qZ9646inMP00pisqvMEZkZhGjEZbXI3qvx4NkIJzD1BrUY2JfKBc22oNzbx/wCrKFVEMrdGt3bb3G9FC7Ktj5byE2+7agmHwbtewPjWzcu4PD47CrI0aXZdx0P/ADVe4ty82G0dgytfbQ2/4pxRLMmxert6n+FNWpyf2m/1H86bproqf8TC3KuX63CHF1JIYbXBVgRfzBtVm5g4b2OIeOElo90YixKsMy387Ee+qhwVrYiEj/EX8xWhyM+KkkKEtkIHecMwGqhdACLZba7+dTOu2C3EqK8OlcnKpNt7V0/B8QL3ie3pV64Pw6ZCTYDTxowwky2t0Pvrl5P2jGVKi48Vq2ZRh8Ob2KmlJwLEXsIn+FXaPBSX1w/Xe9H1ikIII0KkaHXUWpz/AGjHoceK+zL4eEzdmGEZy5rFxY28djp76M/svBTM4hnZCgUMsliSTYMwtuBc3A2yHxFAeG4VkeSJgWujd3XRhqCQD0sfjUeLDsGiMCs7swFiM5JuDbL1udK3sWFIKY7l54gbspt4G9/Ag9QRrfzqdwLkvEzL2rx5Ible0YgBT0JG9r6bdaMoiqJUxAeKSGzMCpB72xUG91vsbnQbmvTzOhjnjMkuSRB3ABlZh1NxpfS9rVPJ5EvijP2BcPyqwIaWZEGYobAtlYC6g+TdD5iiYktOC7qo7M5b9ToLDzqdwXnYQtIJIiyvYkWBOYW1pnivLs3EMQkkSHDwEEntLA6m4KqNfjRxR5JdmvIoR6/MhwyPAiSpYMe74jU/wqXiZ2fFFmOvZ/rUjF8uT4aJs6/WI0YMpQgMqgD2l9b6i9Bv7Rxh8/YEkrb262cWuzGzwKcsp/zmvXwh+phr7Lf4n+dN/t5QumHFmJJBc14eYz2fZ/V48trWJY/1tRQEocLGlyt7ClUBuaGO+Hi/3fxpUsQ2VXg6rJKFkJAsTp42rbuQ+XzhYlld2eSYKTmt3bKbAHfY1nHLXJXa4mSJMTEzRrmGW9yBYnTw71vca2DE4nso0aRwqrYXt1tU9s0xaQIxbnsp/PEn8hWU8dxbzstnCopY6G3XLr41q2KwqSYWWRJCy53kuBbVVvb5VjfBX+zLEXJf4bMTbwvWkI7HGNuiVw/7BWmaUl01UBiAw6D362rS8NIzxGWxIyKSb33A3O9Zrx5c2HUkKJCwFgdwNtBWs8i4dTgYu1ADFMr6jWx7t/dVckQ5IYtIrvEuJLFmuMwLDT3Vm3GuIJI/2aZQpPqfG9b9ieXcNMjKQgboRa400NY9znwUnG/V4kRGCXKlggJ62LG2u/xrnwjGWTM8e3Yd5EZJMK7qgUowDeDX60bUhbgaXqBykGgwLYeXKHZwyqGVjpvqpIrvEz1DmnLXQt2Eo2YhGCkgG1wLinIi3RW38Kn8nf3Sebtc9DRzimLMGZ1izqFubMqnwsL71UKaNJwxKJx3EzwxyzorAqujW2NZHLjXZ+0Zizk3LNrc19D8yYJ8RgcUpXI1iVXMDsLjUV85Rr4itYktG+cncUQwJHdVkCi6ezr4gHcelP8AMkBeEtJrbS22+m9xb1uPWu+CrhzhYRKqOwjUAkAnahfNsgGFk7Gym3Tb330rKXNGLofxMQnPeb/UfzpkHWn8X7ZPjr8Rf9amLgk+pNMfb+sBB/pCXYfF1+FaZUkEl8nX3ImDkyyI3g6n4EGtk4fFgAFxQJE7lllGYgsFPefIDlNiAT6+6sawY+0jA1OdfzFaNwvHD6qMkamQRSFhl1QzR4iYHPclrrHGAD7Pneo5XrZfFG0aBkw17CXU+e9MxLAc2SbUG1r9aE8YhTDRQwzXUlFW8VnYlVsTe1xe1ReUcOkuKVkX+5Ut9opDOdtfdR4OOhXO6LO2AQd0zAHfcdaDc3t9UwjzLLmbRUGm7H9NT7qgc58XEmJjEwdo3FgYhkUW3FyL3GvWq9zpio50hiWQgpqb69LLfxNr0vDxrtBlN9B/kEZsMMQ4LSuzHP5Alb+bG2/ha1uq5QwTQTGchS+mV7XKk3z2G1tba66HxqTyhhzHgokOtgTe34mZv1rvFYbLIGjTUyWfvlNOpyjRj612RxlTM9q0GeYYF4hFIpsuIWJuzYaCQL38jD3GxG1zWW8J4d2j2eRIgBdsx71vIda0vg85E+pC5bZW6X10J6eF/OqVx7laKGeeNbrlJKX3ykZkv46ED3VMmk9CokLxPBQH7FO0cf8AUYXN/K+g91G+F8Td4xIDuTYdNDaqlBy1dVcym7a7C21GMJOmEhVJZVDAkrcgaXvtVcc70DRYk4kbEgmxGqnw661m/MWCEOIdF9m919DqPhe3uo5xPmPDumTDvmmkYKE6Aki7A+Fr0N4/hZpnSVELgxrcgg2IuLb+nxquRpolIESeytcjausTDIoUNG4sNbqR1pjtRl99YjPKVN9qK9pATuB8PZJozDi0CdoArsWS+t2UEAnvAWt1uKt/FJcSkbSyvDIkLKrZnYpJnv2bgW0tcD1FUVuFzAMI4pwhbMrBGJIvdTmA8LEEVasVxyeeGLDfV3gSGMDOUdczKN2IG50OvW5oyjXZpjJdJkrhOIeaEIuLghGVwy5nOck2tlNraeFZxxDD9k2QPcAEErpfXX1rSOBcHV0aSecogjvE0b5jIVYAhVtdiLjS17HSqpzVhZMVjZGCMqMbI3ZsUsO6pJUaXtvSyT9jxl9DjlCaFplE6XiiVmDZC5LaFAQNbEitK45w7sSJcVi5oFlOZYkVSqDQ5Qct+tUfk8thBixJh3kZoyqWvlJva6tsd7i2ulFOGcBkxSRtjZ8aVGwPeyDSwJKtlawvYjoL2pOSQsZPsPzNhIBDLBMzuJBcyEksuViq28SwGg13qp818JxWNl7dMK7u1gNslhc7Gx60S5j4QuESJ5okkiZtFTPHKGA7hYiRldTm9oam+grnljipMeMULNDHbtHcZAYwD7IdzZe7uuVjfTzpcbyt7KqK/iOeXsRjMJBPHiOH5UK5lkRVQKw6uQcuW3U2talPilmTcyKbDuOBYkbAqLsAVOp/WqhNxGF88Rlk7J3Us8l3lVQ1zZQQp0APSiXIUaieWXUYdR7LEd47oCRe2lyT0F/Kk4rtLZNRT0EIsdPCoWKCQhb5vaygA3BFzYm2vjRHsVxsEkszPHBEU7RrjNmOgXe4B6CxG1PpjsJimUTwNHGrE5u1LA3JB7q7i1q7+kLiEGG7GDDQJ2Mo7SQFSqS5e7Hf8Vr38tKpL7IfklWN6DYwuEbsUXEYqF5gOzjnZZFe+iZ8veAO171SuIczyxYpof2dhhJCzR5ViZgSNL20uNLg+dB14zGJY5lwyK0ZUqAz27puvXSxHSrdzzzhcBxBCs00Sq0guXAYZjY6WIsLHzp4oFyySpMPtIiJHLi5o4wR7JVYQjEXsOpPrUbiOP4ZLG0ZxcdmG/arf86zDjOPklijz7A93W52rrhWHiRQ8neJ2ujFV6+FmasZxink1bNOOKnKtfiwXxSALK6owdQSAykEEAlVIPmoB99PrKfqpjuNZM2XML37ova/gvyFGIuKQk27Mr5mNbfK5ofxfiKv3IlAQbtYAsfhoPKhTk3VGk+KEU5Kaf2IXC4DnLEHuRtJvbQDoff8qtHIfGZhJGjzBY86qVFgxAXLckLcgA5dT194AcEcCUk6/Zso9oi5BOy6nYnptuN6u3K3Kc8WafDpnZ0YRAuGiZT3SS2TYBvEnp51U45qvZlxyxafrsZ4ripom7fEYeYMxvHIkiOhFit9CQtyL1J4HicRijK2DR0k0+0eZQVva+liDexO3Wo3HOJoMbkxOGUghEl7LuAgA30UkMRmBBPhTeCxpwGIlaEqiObRro5cDUFipIW4vrbpXRODjoy45q7YO45iMTHI8WIkaQxtexbMpLD0Hjelw9kxM6ZIBGVsrgMWzGx73e2uelGBjcRisRIUTClpmFlkXtLaZQASPnagnBZXw3EUikCxlGYSjQDuhjqfXb1FYyjkqKyp6NQwumZRspVR7v6FR3Gvnf8Aq1UDmDm2eDFPHh37id1gwDAybyHQC1m7v/kqA/P2MtYdkD45Ln5m3yrVSS0QzSOMMYcTg42cjtYpWCgfeJBFz1Nohbw18TQ/6QUnP1adAzZoij6adxjlN/RiNfAUL4Li5MTHBPPKWMTk5mtYDW9hso8h4VbcfzEGjiGFkWTICHsQy66gEEXDbnfqPcm9AZ/HzC6hQ8fs0B5vx4mkRwDbLbX1rTf2rBJ3cRhYyfFe438/jVS+kHhmFWFJcOzBs9ijL0IOtxpuB8agCmcJkYTIV9oHT1savPAI3SMqxGbS9j46/rVR5b4VLiJikSlmVSxA3tt+ZFEsThcRESDnU+Go+VMaLTJjXEoAY+yTa9+vnTePnshYqjEX9pQetVhOLSKbnU2tqKen49nQqy2v1FABhFQgHsotR+A/xpVFi4tDYa9PClQAPi5sYtneLDlrk5wJFa5Ft7nYbDYdBoKL4HnCLLkbDoVVLC0jk6aqt5AO7m1JLeNV/nzhbQYuRstoZmaSFgLIyk3sPArexHQ+RFxHDQC4BFwWFx5dax8UX2jbz8ibSZoGA5qTDqFODKu12LNKkakMbhlXKbAgDbTQb2priPNyzMAEsApAVcQ41a3eusAuR01I1OlA+LypNjbAExoqoAd7Ithf3mh2LwUkUzWjkyq/dIViLX7tjbXS1X+6Rx8te6D975P5aei6ycYlEUhVQoPdBWdprFADl1UWNtzub60LwnEJobEMb6Mw2DHex8rXFQeHMfquJEgAHaiRbtle5ujAIR3ulxfT3VBxWMGXLmN/M+7pXPi06oryOW2y0858fbGxQxpG4khU5kjcsoDlMpsv3roxy/d3rrg8k7o/D8bG8kLEd4KSYyWOU93TKSbix0J+FM4fiiO6ZSqlrmy5xtlJ9oG4HSiknHGjVVXEPZlJcx51FiuUgqdC50udrjqNTvFUqMW72EcfF9Uw00TWxEMoum143sQrm2t7WG5F1tbWo/MHBI8PhY5sO0gSWQLJFL/eRuFLFcwADLpobDQqdb6Q+PFwEn7NlbM2Zr5kNrZAT+MWcG4FxawtTvGODYsyNEkMzd5WbMPvFSRa9rd0geGnlTXYdJkHhSTyyCKAM7tso8tT8qtH0o8SaUYQMACkCqV8GBKv8x8qY+j7hk0HEYjiIpIxlcgkWDXU2sRow32vUDnOMMkktsrLO6FSDmtfNmzbEZmK237tVYY/GytiWik6TYpo1SNnNlXQeVhrsNOpqv56v/K2OnjwySRWEX93IQVuSCSuh1OhNaccHySUV7M20lbAPFcOBArk98ysuXplUWuP/MCL1yeZcRkSMuMqZct1H3RlXU+ArjizqDLFYF1nuHA+7ZhlF9QASNKXLnGDhJxOqByARlJsNdNwDUtU6Y10eLzBN0df3V/r/mosuKJFssY/0xop+IF/+BWi86cROI4TFOVCl5FOUG4Htjf3VmdJ6GHOTMB2+Lihvl7RspNr2ukmtWhZsCuaD9pSRlEytngGViBqoyTXW7fdYdPKq3ySk31mNsMgeWMlwh0BAGQXP3ReS1/E0e4pwm8rtiOH4iN2Jdjh5Y5xdiSe40RG99A4tURnU+/1LlH4p/8AAXjOHcJaCJlxcoxDXMoyrkBJFr3e6qNdVzkgbA6VJj4TwjKFPE3zAWDdicg3Nst8xFzv8qFycDmkVpcMc8IbLdssLqbXsy5yoNr7Mdj4EULx2CxMX94rgeIbOvjupIrRyszosOAxMUTIYsRCrq+YSguGuNic4tbY2/nUXjOKMPEZHxKl2dftMwsSXF85UAaey1gKicrxy/WsOcjnMSVvezABlOvhfSmeacKkWIVQSUMcRuRY2Ki9tToNQPSpbT0NWnYNxuILyO9rZmLW1Nrm+539aaGu1Jjc2XUk2A9dqncZ4f8AVpuyvchUa/nlBb3Zr0xFq5YwLnBSWY2kbRRuACA9v9Vj5VaoUAVQI1SygEKABfqTbcnqahcEYDDxea3/AHiWH51LEw8RUtbsq9UdyRhhYgGqzzvhrYYlb2DLp8v1qxmdfxD4ignNOMQwPECCzDprbX+VNCAXJMjJiHeMsD2SklTY62v/AMVfl5lkYZJezlHhKov8dDVa5R4bHDD23aIWkOUrmGZQviPPejE00TCzFD7xQwQ7iIMBL/eYdoyesT5h8GofNyXhZP7jGID+GUFD8djXLqg9iZfRiCPje9NyYxFHedPcbigBh/o4xd+6EYdCHWx+dKu/rkX4k+NKgA5LzRgDIImwz6tZkcKUN9NY8pDb/lVhx/0e8PZswhEZHVGZP9oOX5U5wbmPCYmDDy4zDx555zHEVSzXDEJqu1svteVSOJTwDK0ryxRFnTtO0BAZZTGAc12syBn0sO4fEVhyxlKnF0zfhnCNqW0Z9heUmfGPJg0MsYuriUFDGdD3rgXuDoQPukUI56jnwmKF2UZwrqI5cyjLYZSo2sRex8RRPmThfZtKMPNJlRVeZWcZo824bVVezXBzEEXHQ3FL41KrZWV2ZbDKXQRttc90E6A9QSPPWuiHJLxKDMuSMc3KPQzieLyMCo0B31Yk9dSxPU3qAig3ubaaaX93lXNeUiAlwvEqikNezAghTY9bD4n5U7AqdwlwGUFgN72ud/usQO7cb2vYWNCVcj8uh/OpDThzdrLoB3VFtAF28wLk+JNIa7C+BxUbiJGVV+0DTMS5ditzrc2IbbbQk+NWXnRppHTGQoXXFRgPbXLJEOzfppdcjafiNVfB4NSqt7S27zrZbWLaa+VhfyrRODRdrwk9qiOsb9pExAaxB7KQMp27puL6G3kQJbaaZoktoz/gvMGIUhUlsqbK8ioAdu6XtlPpbrRQ46aKJUxCi8rNIcwDKwzXFjqGF9bjxFLHYKUDNG6n0VQf3SCPDQVL5InXEnE4TGv9mYTIjBR9m62UOoA0aza29q1j52ZtnOG5igQssuHlIsuVoQljcXOjJ6jQnanMLzSHLRpE0cftDOy+moyqM1yNr6A+tDMHwnJhzPiGCgkZYtc7b97TQLoRvrcVCfimGXfDuPNZjf1sykfGhKndibDHOfDYlw8OJQr2k0jhwCDtcAn90/vUA5f4Q2KnWBWClgTcgkd0X6elS+McyriMNDhlVwIGZgWIN8177De5oIt76b0/YGv47lOR+HRYLtFDRkEvY5TYsdt/vVlPFcEYJpISQxjYqSNAbdaZxCsjMjHvKSpF72INiLg2OtN0N2Bsn0GYSMRS4gIGlTMNEu5XwzX1FxtbcDyqDzlzfJM86/V4HWFrJdWz7DNZgbDUm48tbaXifRrgAMLLipXeOKIm7o7oxI1sMrDXYCgnMfMWDd2k+rO7u5L9ob30AvmUqQdNR49elR4l5M39KKz+OJxyrhZ53mijxCwzSMzhWJKuSCSAFub2LeYFzY1C4nFi4HYYtLhDqM+ViDoGBvfKdxp+tDYuKQLMs6pOrqbgrIBttqQTa2m+1WPGc8YaYgvgVuq5Ba4NvNy5N99d9a0pEp/Ul4bjKxxLEvD8QJCCVLYjb/NcpcAHXoPOoh4sqgNmjFlVcnaGVhprZrZct+gbS4HnXvH+PRTQd4MskgjEcSn7NQbmR2a3fI7qgEdb1F4dPCoHcUnxKgn+PjUJJDbJf9pul6tvJqQY8tBMquGRtwLggXBB6EdCKp+O5gSOwWJGO+qiwBsffe21SuA8xZJBNEkUcgFu4uUa76DQnztTAMcqPBhGVHEcmbS8gDBRsoF7helzVu5s5UhxUDSRxiKVFzKyABWAGY3y6HS9qy7H8Ojcd0sD4anfXf1q6ckcbliweIjllDL2ZWMG4YX7p12sASfHTr0lvZWqKaOXf/E+X866HLY/xPl/OrAVArzNVWTQA/s2v+IfhXn9nk/G3wFHwa5cUWFAH9gp+Jvl/CuP2PF+J/h//NHWpmR7dCfSiwoFfsWL/wAT4H/20qI9ufwN8v40qAGeBc2wxpDH2JbsdUvkuG6kE7XJPxohNzLBOvZS53C5TFHYjvhWzyHIb9531BJGVT1NZ9BKbWCgedjeukaUDS6jrbu39SNT7zWbv0Uq9mlYfmnCYdsQ7nNLMxYrux6BdPZG5PhmNthVNl4djZlZQM0NiwLEAWHe07xJ+FVloLDNe9j08elWfhPHZrP2IIAXRTqoJGoA3GxsB76XFx4Ntu7L5eXNJJUkVICvBTjoVbKd10NN2rUxPafwmGzmmBRPg6nMLDc0APTcKhQoWdtdSABpoCNfO/yooeNBIpIos1nULbYABgx06+zb31G5jSxie1s6a+osb+XddNPKg4apxvsrJroLwcSOgJ0/r4a1Kw1lxc1mAVoRcjpnMZO/+qq+GqYmPKmSYAFsoBDC6kB4xYg9LACqJDnGeKxzBYWAQxexmyxsbjq/fF9TobW2oCmAvcrhzJ5iTtvj2O1ROITguWRAEbUIbMFuASoIAIANwLW0ApjJEw0zI3h7aH/1L/uoGOvoSphEZHk4b/exrwG2tOuxygFy1vFibeGh269K8w05Rgw3APUjcFd1IOx8flQIP4/mXEdpIwjjVc59vDQgrckgOSps1t7noaB8QLmVzILOWOYWAAN9dBoNfDSrZ2BmjeReEyNHK3ak/WGBYjNZlXLmI77eyLG9VTiWJ7WV5MuXOxOW98vlewvamwC3A8RZAkkswh7xaKNhlZr90lXunTU5b6Vzx3jMvaG9mW5I7aOMybkXZlVbtYV7wTByPHeKRlbOAFAGoNwTmP4Tl0/zV1x6CazFpZHyHvu1ktfRQDm7199akpNroD4WUG4MEYUkXP2oHr/eWFeskO5Rl/0S5gfS8Z+bVDw6XDbsfLXxO/x8dqcTEbA6Dp1PxP6Wpkj+PmQ9mFDCw+8QTbUdOt687SoitfW2w0/et+tOX0Nt7fyoGdYqbMxPTp6DQfKm4ZipuDXDV5QIL4LiTFgMxA6mizYsMDEp7htdSdD76qiNaieGcEXXW3gdfhvSGi+8KmHYZD91lCX3Fw5K+ndBH86dYgVTMJxIieGHNZTJHmY6WubEnwtmN/StPPAof+9w/wAaEMA3pXFHP2FAP/noPif0U143C8Ep1xjHzVUt7ryA/IUAAjTbEVYPqeA/7mb92P8A91cHC8O/x5/3V/nQBX7jypUcHD8B/wB2w8sr/wD66VAFWw+JjhUI+ChkJ+9YrIeu62J+NeGfDyRsUw0CXzW1LuLdbM5tdiALjW99gSJK/RdinN5sWovvZS3yuBRLB/RNALdriJX8QoVB881TaCmQcOuCTIZMYtk/6bRAC43HZpmup6G+u+1jVclaKItHhMQrRyDvNLGy21v5kEZR3l3GnU1p+E+jfh6/9Fm82kf8gQKe4vwTheBgbESYVMqW01ZmJ0VRdtyf49KM0GJhXEU+2ZVdX1AzKCqkkDYNqNfGo80ZVipFiNwd6IYnHGbENiQIoiGDqmoUZbZQBrc90eutDZHJYknUm59Tqask8o1wDFBWsQDQWpnDWbMAu5/rpQAb5rnB7JR90N+SJ8Ps6r+ainM8hLrf7qqpPQm1/jre3hQ4Ya6K4YC5IYNpYix08rMN+t6BnAenlcGJlAu1ib36ZhcW8dL389jpUnh3CBKsrCU5owCEWN5C5OgF0vl16kfwo9gOAKvdMb3LAdpIyYePbM984aQAeyNN97Hu0UIrLSuTfskvYC4BOgAA0zW2A6VyLjUAg+UKX/OrzhcHFnMrHCwrGGHZktimci+uUSWa4tbbfVba1Xkwzk3M6oT90CMH5kU6AESSu3ttIR/mFh+Z1ppxcEUT4pAyqM0zPrsb2G+ugy39/WhbbUMEHsXHhZnOIbGmMscxRonaVDvlVgcpA2U3FgBtQriWI7SWSSxGdi1jvqb6+Z3PmTV5l7WKHMFTDyWuPqkCz5vHMclkP/3D6VQ8XM7uzSkl2N2LaEnz0FDGO4PBNJfLA8g01Q2I30uQR46WvtqOs2LgeOUACHFC4uLE26jp6GpfLSqV1nmjIY6REqbECxLBGG4O9raeNWyMFB3MbiR5tLh2HwcA0gop8XCOIAg9nNfoSEv8WOlMYnguMGZ3w4IAuSVUk/uG5OvStF4fxfFIRfEpIl+9ngil0tuMki7XB0IOm9HcLjJ41hv9SxGeR1uGlw5C2Zoy1w+QGwF2UAEgXNwSAYEGJBNtDoLeRUka69RUjCS2Elxe6W99xb9a1TnKXDz4YyYjAPE4ygzDs2VW9khngOZdTYZ1YHTxBrNsZw4JneMs0F7I8gCF/Ahb3/lSCwWxrwV1mzHbSuHGtMQgaL4OQZQevw/Sg96eje1AInSSFsTGbWKlLkm46EdB0I99aHItVTgvBBMBM7WAb2RswUADX1Hyq1MfOkUN5a5Irpx602aAOWqHPipNliJ8CWUD86mW9a5IoAG/WcR/gj94fxpUQuKVAjWVsOlOowpUqyLHbgAk6WFzVI+kHhsmPw+SKy9nIHXMdXYBltpoujEjp4mlSpoGYmgyMysoJ2sdbGoqtSpVqZs7D1JwOKZHDKbHxG9KlQAY48qns5TbvKVJ13Gvkb2I1vQdcWiiwS/+3x9fGlSoGL9quBlUkLe+W7Wv42va/nauUxEj7N8h/DzpUqBHDYqX8bfH+FJMfIP+pJ7nIpUqB2PGVyO8XtvYtmFO4HCNKxVbXCltetth7yQPfSpVM3SbBHGIi7KRlVtVNsw7p8/Ma/lTUkhYlmJJO5JJJ9Sd6VKnHoZxJiXWwV2A3sGIHypLxKYbTSD0dh+tKlTEe/tKb/Fk97k/nXacTm/GT62b8waVKgQ9iOIO6KH1sdBZR0/ygf0ahzTvJbOxNhYXN7DwHhSpUAdxWG5puY6j0pUqBHFIeNKlQM0Xl9mGGj7pOl+nUkjr4GppZvw/Ej9KVKkUNsx8B8T/AApslvED3E/rSpUActm8R8P51wxb8R+X8KVKgBW82+I/hSpUqAP/2Q=="
    ,"car_focus": "การพัฒนาเพื่อทวงคืนตำแหน่งแชมป์โลกทีมผู้สร้าง",
    "standings": {
        "Max Verstappen": {"pos": 3, "pts": 230, "nat": "NED"},
        "Liam Lawson": {"pos": 15, "pts": 20, "nat": "NZL"}
    }
}



In [128]:
# ===== List → string สวย ๆ (รองรับไทย/อังกฤษ) =====
def _stringify(val, thai=False):
    if isinstance(val, (list, tuple)):
        if not val: return ""
        if len(val)==1: return str(val[0])
        conj = " และ " if thai else " and "
        return ", ".join(map(str, val[:-1])) + conj + str(val[-1])
    return str(val)

def fill_response(template: str, facts=FACTS):
    th = bool(re.search(r'[\u0E00-\u0E7F]', template))
    def repl(m):
        key = m.group(1)
        try:
            v = eval(key, {}, facts)  # รองรับ {drivers[0]} / facts["standings"]["George Russell"]["pts"]
        except Exception:
            v = facts.get(key, m.group(0))
        return _stringify(v, th)
    return re.sub(r"\{([^\}]+)\}", repl, template)

In [141]:
INTENT_TEMPLATES = {
    "greeting": {
        "questions": [
            "สวัสดี","หวัดดี","hi","hello","hey","ทักทายหน่อย","เริ่มกันเลย","ไง","how are you","greetings"
        ],
        "responses": [
            "สวัสดีค่ะ! คุณสามารถถามเรื่อง {team_short} ในฤดูกาล {year} ได้เลยค่ะ 🏎❤️🐂"
        ]
    },
    "greeting_e": {
        "questions": [
            "hi","hello","hey","how are you","greetings"
        ],
        "responses": [
            "Hello! You can ask about {team_short} in the {year} season 🏎❤️🐂"
        ]
    },
    "goodbye": {
        "questions": [
            "ลาแล้ว","ไปก่อนนะ","bye","goodbye","see you","ไว้เจอกัน","แค่นี้แหละ","จบการสนทนา","ไปล่ะ"
        ],
        "responses": [
            "ไว้คุยกันใหม่ค่ะ 🏎️💨"
        ]
    },
    "goodbye_e": {
        "questions": [
            "bye","goodbye","see you"
        ],
        "responses": [
            "Talk to you later 🏎️💨"
        ]
    },
    "capabilities": {
        "questions": [
            "บอทนี้ทำอะไรได้บ้าง","ช่วยเรื่อง {team_short} {year} ได้ไหม","ฉันถามอะไรได้บ้าง","บอทนี้มีข้อมูลอะไร","แนะนำหน่อย","ถามไรดี"
        ],
        "responses": [
            "ฉันตอบคำถามเกี่ยวกับ {team_short} {year}: นักขับ รถ ทีมงาน ฐานทีม คะแนน และบริบทฤดูกาล (อัปเดตถึง {asof_date}) "
        ]
    },
    "capabilities_e": {
        "questions": [
            "what can you do","what can I ask"
        ],
        "responses": [
            "I can answer questions about {team_short} {year}: drivers, cars, team, base, points, and season context (updated until {asof_date})"
        ]
    },
    "team_overview": {
        "questions": [
            "สรุปทีม {team_short} {year} ให้หน่อย","ภาพรวม {team_short} {year}",
            "ข้อมูลโดยรวมของทีม", "ภาพรวมของทีม Red Bull ปี 2025","ทีมเป็นไง","ขอรายละเอียดทีม"
        ],
        "responses": [
            "{team_full} {year} ใช้รถ {car_name}; นักขับคือ {drivers[0]} และ {drivers[1]}; หัวหน้าทีม {team_principal}; ผอ.เทคนิค {technical_director}; ฐานทีม {base_chassis} / {base_engine}. "
        ]
    },
    "team_overview_e": {
        "questions": [
            "Team overview of {team_short} {year}", "Team overview","overview {team_short} {year}", "overview"
        ],
        "responses": [
            "{team_full} {year} uses car {car_name}; drivers are {drivers[0]} and {drivers[1]}; Team Principal {team_principal}; Technical Director {technical_director}; base {base_chassis} / {base_engine}."
        ]
    },
    "drivers_2025": {
        "questions": [
            "นักขับ {team_short} {year} มีใครบ้าง","{team_short} {year} drivers","ไลน์อัพ {team_short} {year}",
            "คนขับของ Red Bull ปีนี้","ใครเป็นนักขับให้ Red Bull ปี {year}","คนขับชื่อไร","นักแข่งชื่อไร","ใครเป็นคนขับ","คนแข่งปีนี้",
        ],
        "responses": [
            "นักขับปี {year} ของ {team_short} คือ {drivers[0]} และ {drivers[1]}."
        ]
    },
    "drivers_2025_e": {
        "questions": [
            "{team_short} {year} drivers",
            "Who are the drivers of {team_short} {year}?","Drivers lineup of {team_short} {year}"
        ],
        "responses": [
            "The {year} drivers of {team_short} are {drivers[0]} and {drivers[1]}."
        ]
    },
    "drivers_standings_2025": {
        "questions": [
            "อันดับนักขับของ {team_short} ปี {year} ตอนนี้","คะแนนของ Max Verstappen ปี {year} เท่าไหร่",
            "คะแนนของ Liam Lawson ปี {year} เท่าไหร่","Verstappen อยู่อันดับที่เท่าไหร่ ปี {year}","Lawson อยู่อันดับที่เท่าไหร่ ปี {year}",
            "สรุป standings นักขับ {team_short} {year}","อันดับของ Verstappen และ Lawson ตอนนี้",
        ],
        "responses": [
            "อัปเดตถึง {asof_event} ({asof_date}) — Max Verstappen อันดับ P{standings['Max Verstappen']['pos']} มี {standings['Max Verstappen']['pts']} คะแนน; Liam Lawson อันดับ P{standings['Liam Lawson']['pos']} มี {standings['Liam Lawson']['pts']} คะแนน. "
        ]
    },
    "drivers_standings_2025_e": {
        "questions": [
            "How many points does Max Verstappen have?","Driver standings of {team_short} {year}","point max","max point" ,"liam point"
        ],
        "responses": [
            "Update as of {asof_event} ({asof_date}) — Max Verstappen is P{standings['Max Verstappen']['pos']} with {standings['Max Verstappen']['pts']} points; Liam Lawson is P{standings['Liam Lawson']['pos']} with {standings['Liam Lawson']['pts']} points."
        ]
    },
    "team_principal": {
        "questions": [
            "หัวหน้าทีม {team_short} คือใคร","team principal {team_short}",
            "ใครเป็นหัวหน้า Red Bull","ใครคุมทีม Red Bull","Who is the team principal of {team_short}?"
        ],
        "responses": [
            "หัวหน้าทีม (Team Principal/CEO) คือ {team_principal}. / The Team Principal/CEO is {team_principal}."
        ]
    },
    "team_principal_e": {
        "questions": [
            "team principal {team_short}",
            "Who is the team principal of {team_short}?"
        ],
        "responses": [
            "The Team Principal/CEO is {team_principal}."
        ]
    },
    "technical_director": {
        "questions": [
            "ผู้อำนวยการฝ่ายเทคนิค {team_short} คือใคร",
            "ใครออกแบบรถ Red Bull","ใครเป็นผู้อำนวยการด้านเทคนิค"
        ],
        "responses": [
            "ผู้อำนวยการฝ่ายเทคนิคคือ {technical_director}."
        ]
    },
    "technical_director_e": {
        "questions": [
            "technical director {team_short}",
            "Who is the Technical Director of {team_short}?"
        ],
        "responses": [
            "The Technical Director is {technical_director}."
        ]
    },
    "bases": {
        "questions": [
            "ฐานทีม {team_short} อยู่ที่ไหน","โรงงาน {team_short} อยู่ไหน",
            "สำนักงานใหญ่ Red Bull F1","Red Bull สร้างรถที่ไหน"
        ],
        "responses": [
            "ขอโทษครับ/ค่ะ บอทนี้ตอบเฉพาะ ทีม Red Bull ฤดูกาล 2025 เท่านั้น"
        ]
    },
    "bases_e": {
        "questions": [
            "where is {team_short} based",
            "Where is {team_short} headquarters?"
        ],
        "responses": [
            "Chassis: {base_chassis}; Engine: {base_engine}."
        ]
    },
    "car_name": {
        "questions": [
            "รถปี {year} ของ {team_short} ชื่ออะไร","ชื่อรถ {team_short} {year}",
            "รถ F1 ของ Red Bull ปีนี้ชื่ออะไร","What is the name of {team_short} car {year}?","ชื่อรถ","รถชื่ออะไร"
        ],
        "responses": [
            "{car_name}. / {car_name}."
        ]
    },
    "car_name_e": {
        "questions": [
            "What is the name of {team_short} car {year}?","car name","what is car name?"
        ],
        "responses": [
            "{car_name}. / {car_name}."
        ]
    },
    "max": {
        "questions": [
            "max max","tututu","maxmax","supermax",
            "ขอเพลงเชียร์max"
        ],
        "responses": [
            "max max super max มาเชียร์กัน {max_song} 🏁 🏎️ 💨"
        ]
    },
    "max_e": {
        "questions": [
            "maxsong","Cheer song for Max","max cheer"
        ],
        "responses": [
            "max max super max let's cheer {max_song} 🏁 🏎️ 💨 {max_picture}"
        ]
    },
    "win_score": {
        "questions": [
            "ข้อมูลชนะ","maxชนะสนามไหนบ้าง","maxชนะที่ไหน","สนามที่maxชนะ","รวมสนามที่ชนะ"
        ],
        "responses": [
            "Max Verstappen คว้าชัยชนะมาแล้วหลายสนาม เช่น Monaco GP, Italian GP, Abu Dhabi GP และอีกมากมาย 🏆 ",
            "Verstappen ชนะในหลายสนามทั่วโลก ทั้ง Monaco, Silverstone, Suzuka, Austin และสนามอื่นๆ อีกเพียบ 🌍 ",
            "เขาเก็บชัยชนะรวมได้มากกว่า 50 สนามใน F1 โดยมีทั้งยุโรป เอเชีย และอเมริกา 🏎️🔥 ",
            "แต่ในปีนี้(2025)เขาชนะไปทั้งหมด 3 สนาม โดยล่าสุดคือชัยชนะที่สนามโมนซา (Monza) ในรายการ Italian Grand Prix เมื่อวันที่ 7 กันยายน 2025 ซึ่งเขาได้ทำลายสถิติการแข่งขัน F1 ที่เร็วที่สุดในประวัติศาสตร์ ด้วยเวลา 1 ชั่วโมง 13 นาที 24.325 วินาที และความเร็วเฉลี่ย 250.706 กม./ชม. "
        ]
    },
    "win_score_e": {
        "questions": [
            "Which races did Max win?","Max Verstappen victories","max win", "history max win"
        ],
        "responses": [
            "Max Verstappen has won multiple races including Monaco GP, Italian GP, Abu Dhabi GP and more 🏆",
            "Verstappen has won races all over the world: Monaco, Silverstone, Suzuka, Austin and many more 🌍",
            "He has collected over 50 F1 victories worldwide including Europe, Asia, and America 🏎️🔥",
            "This year (2025) he won 3 races, latest at Monza in Italian GP on 7 September 2025, setting the fastest F1 race record with 1h 13m 24.325s at average speed 250.706 km/h."
        ]
    },
    "who_win": {
        "questions": [
            "ทีมเราชนะล่าสุดตอนไหน","ทีมเราชนะกี่สนาม","กี่สนามเราชนะ","ผลชนะปีนี้"
        ],
        "responses": [
            "ในปีนี้เราชนะไปทั้งหมด 3 สนาม โดย Max Verstappen ล่าสุดคือชัยชนะที่สนามโมนซา (Monza) ในรายการ Italian Grand Prix 7 กันยายน 2025 ซึ่งเขาได้ทำลายสถิติการแข่งขัน F1 ที่เร็วที่สุดในประวัติศาสตร์ ด้วยเวลา 1 ชั่วโมง 13 นาที 24.325 วินาที และความเร็วเฉลี่ย 250.706 กม./ชม. "
        ]
    },
    "who_win_e": {
        "questions": [
            "When did our team win last?","How many races did our team win?","last win"
        ],
        "responses": [
            "This year we won 3 races, Max Verstappen latest at Monza, Italian GP on 7 September 2025, setting the fastest F1 race record 1h13m24.325s, average speed 250.706 km/h."
        ]
    },
    "car_focus_2025": {
        "questions": [
            "ปีนี้ทีมพัฒนาอะไรเด่น","โฟกัสของ {car_name} คืออะไร","แก้อะไรจาก RB20 เป็น RB21","Red Bull RB21 มีจุดเด่นอะไร","What is the focus of {car_name}?"
        ],
        "responses": [
            "{car_name} เน้น {car_focus}. / {car_name} focuses on {car_focus}."
        ]
    },
    "constructors_standings_asof": {
        "questions": [
            "อันดับคอนสตรักเตอร์ของ {team_short} ตอนนี้","คะแนนทีม {team_short} {year}","อันดับคะแนนทีม Red Bull","Red Bull ได้กี่คะแนนรวม","Current constructors standings of {team_short}?","How many points does {team_short} have?"
        ],
        "responses": [
            "อัปเดตถึง {asof_event} ({asof_date}) {team_short} อยู่อันดับ 3 มี {constructors_points_asof} คะแนน. / Update as of {asof_event} ({asof_date}) — {team_short} is P3 with {constructors_points_asof} points."
        ]
    },
    "results_live_disclaimer": {
        "questions": [
            "ผลล่าสุดวันนี้","ใครชนะวันนี้","Red Bull ชนะไหมวันนี้","คะแนนสด","ผลการแข่งล่าสุด","Latest results today?","Who won today?"
        ],
        "responses": [
            "ขออภัย ฉันไม่ให้ผลสด แนะนำเช็คที่เว็บไซต์ F1 (ข้อมูลของฉันอัปเดตถึง {asof_date}). / Sorry, I cannot provide live results. Please check the F1 website (my data is updated until {asof_date})."
        ]
    },
    "out_of_scope": {
        "questions": [
            "ใครชนะ MotoGP","ข่าวหุ้นวันนี้","ผลลิเวอร์พูล","ผลบอลเมื่อคืน","ราคาทองวันนี้","ทีมอื่นบ้าง","ขอข้อมูลทีมMercides","Other sports","Stock news","Gold price today"
        ],
        "responses": [
            "บอทนี้ตอบเฉพาะเรื่อง {team_short} ฤดูกาล {year} เท่านั้นค่ะ / This bot only answers questions about {team_short} {year} season."
        ]
    },
    "season_stats": {
        "questions": [
            "Max ชนะทั้งหมดกี่สนามใน F1","Red Bull ชนะทั้งหมดกี่แชมป์","สถิติ Max Verstappen ตลอดฤดูกาล","สถิติ Red Bull ปีนี้","รวมชัยชนะของทีมและนักขับ"
        ],
        "responses": [
            "Max Verstappen ชนะรวม 3 สนามใน F1 และ Red Bull ชนะรวม 3 สนาม ",
            "สถิติฤดูกาล {year}: Max ชนะ 3 สนาม, Red Bull ได้ 239 คะแนนทีม "
        ]
    },
    "season_stats_e": {
        "questions": [
            "Max wins this season?","Red Bull wins this season?" ,"team victory"
        ],
        "responses": [
            "Max Verstappen won 3 races in F1 and Red Bull won 3 races total",
            "Season stats {year}: Max won 3 races, Red Bull has 239 constructor points"
        ]
    },
    "race_calendar": {
        "questions": [
            "สนามต่อไป Red Bull แข่งที่ไหน","ตารางแข่ง F1 ปี {year}","Red Bull มีแข่งสนามไหนบ้าง","วันแข่ง Italian GP ปีนี้","ปฏิทินแข่งฤดูกาล {year}","Next race of Red Bull?","F1 calendar {year}","Where does Red Bull race next?"
        ],
        "responses": [
            "สนามต่อไปสำหรับ Red Bull คือ Azerbaijan Grand Prix วันที่ 19–21 กันยายน 2025 / Next race for Red Bull is Azerbaijan Grand Prix, 19–21 September 2025",
            "ตารางแข่ง F1 ปี {year} / F1 Race Calendar {year}:"
            "14-16 มี.ค.: Australian Grand Prix (เมลเบิร์น) / 14-16 Mar: Australian GP (Melbourne)"
            "21-23 มี.ค.: Chinese Grand Prix (เซี่ยงไฮ้) / 21-23 Mar: Chinese GP (Shanghai)"
            "4-6 เม.ย.: Japanese Grand Prix (ซูซูกะ) / 4-6 Apr: Japanese GP (Suzuka)"
            "11-13 เม.ย.: Bahrain Grand Prix (ซาคีร์) / 11-13 Apr: Bahrain GP (Sakhir)"
            "18-20 เม.ย.: Saudi Arabian Grand Prix (เจดดาห์) / 18-20 Apr: Saudi Arabian GP (Jeddah)"
            "2-4 พ.ค.: Miami Grand Prix (ไมอามี) / 2-4 May: Miami GP (Miami)"
            "16-18 พ.ค.: Emilia Romagna Grand Prix (อิโมลา) / 16-18 May: Emilia Romagna GP (Imola)"
            "23-25 พ.ค.: Monaco Grand Prix (โมนาโก) / 23-25 May: Monaco GP (Monaco)"
            "30 พ.ค. - 1 มิ.ย.: Spanish Grand Prix (บาร์เซโลนา) / 30 May-1 Jun: Spanish GP (Barcelona)"
            "13-15 มิ.ย.: Canadian Grand Prix (มอนทรีออล) / 13-15 Jun: Canadian GP (Montreal)"
            "27-29 มิ.ย.: Austrian Grand Prix (สปีลเบิร์ก) / 27-29 Jun: Austrian GP (Spielberg)"
            "4-6 ก.ค.: British Grand Prix (ซิลเวอร์สโตน) / 4-6 Jul: British GP (Silverstone)"
            "25-27 ก.ค.: Belgian Grand Prix (สปา-ฟรองโกชองส์) / 25-27 Jul: Belgian GP (Spa-Francorchamps)"
            "1-3 ส.ค.: Hungarian Grand Prix (บูดาเปสต์) / 1-3 Aug: Hungarian GP (Budapest)"
            "29-31 ส.ค.: Dutch Grand Prix (ซานด์ฟอร์ท) / 29-31 Aug: Dutch GP (Zandvoort)"
            "5-7 ก.ย.: Italian Grand Prix (มอนซา) / 5-7 Sep: Italian GP (Monza)"
            "19-21 ก.ย.: Azerbaijan Grand Prix (บากู) / 19-21 Sep: Azerbaijan GP (Baku)"
            "3-5 ต.ค.: Singapore Grand Prix (สิงคโปร์) / 3-5 Oct: Singapore GP (Singapore)"
            "17-19 ต.ค.: United States Grand Prix (ออสติน) / 17-19 Oct: United States GP (Austin)"
            "24-26 ต.ค.: Mexico Grand Prix (เม็กซิโก ซิตี้) / 24-26 Oct: Mexico GP (Mexico City)"
            "7-9 พ.ย.: Brazilian Grand Prix (เซาเปาโล) / 7-9 Nov: Brazilian GP (Sao Paulo)"
            "20-22 พ.ย.: Las Vegas Grand Prix (ลาสเวกัส) / 20-22 Nov: Las Vegas GP (Las Vegas)"
            "28-30 พ.ย.: Qatar Grand Prix (ลูเซล) / 28-30 Nov: Qatar GP (Lusail)"
            "5-7 ธ.ค.: Abu Dhabi Grand Prix (ยาส มารีนา) / 5-7 Dec: Abu Dhabi GP (Yas Marina)"
        ]
    },
    "car_stats_2025": {
        "questions": [
            "RB21 top speed เท่าไหร่","RB21 aerodynamics มีอะไรใหม่","สเปครถ Red Bull ปี 2025","รถทีม Red Bull ปีนี้แรงม้าเท่าไหร่","RB21 specs","RB21 top speed"
        ],
        "responses": [
            "{car_name} ปี {year} มี top speed {top_speed} กม./ชม., น้ำหนัก {weight} กก., และเน้น {car_focus} / {car_name} {year} has top speed {top_speed} km/h, weight {weight} kg, focusing on {car_focus}."
        ]
    },
    "team_comparison": {
        "questions": [
            "คะแนน Red Bull vs Mercedes ปีนี้","Max vs Lewis อันดับคะแนนเท่าไหร่","เปรียบเทียบ Red Bull กับทีมอื่นปี {year}","ทีมใดมีคะแนนนำตอนนี้","Red Bull vs Mercedes points","Driver comparison Max vs Lewis"
        ],
        "responses": [
            "คะแนนเปรียบเทียบทีมปี {year}: Red Bull 239 คะแนน, Mercedes 180 คะแนน / Team points comparison {year}: Red Bull 239 point, Mercedes 180 point",
            "อันดับนักขับ Max Verstappen อยู่อันดับ 3 ด้วยคะแนน 230 แต้ม ขณะที่ Lewis Hamilton อยู่อันดับ 5 มีคะแนน 180 แต้ม"
        ]
    }
}


In [130]:
# ---------- augmentation + dataset builder ----------
EMOJIS = ["🏎️","✨","🔥","💬","✅","❓","📊","🏁","🏎️💨","𓄀"]
PUNCTS = ["","!","!!","!!!","?","??","..."]
TEAM_SYNS = [
    "Oracle Red Bull Racing","Red Bull Racing","Red Bull","RBR",
    "ทีมเรดบูล","กระทิงแดง"
]
YEAR_SYNS = [FACTS["year"], f"ฤดูกาล {FACTS['year']}", f"season {FACTS['year']}", f"{FACTS['year']} season"]

def augment(text: str) -> str:
    t = text.strip()
    r = random.random()
    if r < 0.1: t = t.upper()
    elif r < 0.2: t = t.capitalize()
    if random.random() < 0.35: t += " " + random.choice(EMOJIS)
    if random.random() < 0.35: t += random.choice(PUNCTS)
    if random.random() < 0.1 and len(t.split())==1 and len(t)>3:
        t = t[:-1] + t[-1]*random.randint(2,4)
    return t

def expand_question(q):
    return q.replace("{team_short}", random.choice(TEAM_SYNS)).replace("{year}", random.choice(YEAR_SYNS))

def build_dataset(target_rows=9000, seed=42):
    random.seed(seed)
    rows = []
    intents = list(INTENT_TEMPLATES.keys())
    per_intent = max(200, target_rows // len(intents))
    for intent, pack in INTENT_TEMPLATES.items():
        qs = pack["questions"]
        rs = [fill_response(r) for r in pack["responses"]]
        for _ in range(per_intent):
            q = augment(expand_question(random.choice(qs)))
            r = augment(random.choice(rs))
            rows.append((q, intent, r))
    random.shuffle(rows)
    return pd.DataFrame(rows, columns=["question","intent","response"])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [142]:
# --- สร้างและบันทึก dataset ---
df = build_dataset(target_rows=9000)
df.to_excel("q_a_dataset.xlsx", index=False)
print("สร้าง q_a_dataset.xlsx จำนวนแถว:", len(df))

สร้าง q_a_dataset.xlsx จำนวนแถว: 8995


In [143]:
# =========================================================
# 2. Train, Evaluate, Save Model
# =========================================================

# Load (จากไฟล์ที่เพิ่งสร้าง)
df = pd.read_excel("q_a_dataset.xlsx")

X, y = df["question"], df["intent"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

responses_dict = df.groupby("intent")["response"].apply(list).to_dict()

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.9533
                             precision    recall  f1-score   support

                      bases       0.96      1.00      0.98        51
                    bases_e       1.00      1.00      1.00        51
               capabilities       1.00      1.00      1.00        51
             capabilities_e       1.00      1.00      1.00        52
             car_focus_2025       1.00      1.00      1.00        51
                   car_name       1.00      0.80      0.89        51
                 car_name_e       0.85      1.00      0.92        52
             car_stats_2025       1.00      1.00      1.00        52
constructors_standings_asof       1.00      0.96      0.98        51
               drivers_2025       1.00      0.94      0.97        51
             drivers_2025_e       0.91      1.00      0.95        52
     drivers_standings_2025       1.00      1.00      1.00        51
   drivers_standings_2025_e       1.00      1.00      1.00        51
                

In [133]:
import pandas as pd

# โหลด dataset เดิม
df = pd.read_excel("q_a_dataset.xlsx")

# ตัวอย่างคำถามใหม่ + intent + response (ลิงก์เพลง Super Max)
new_rows = [
    {"question": "max max max", "intent": "max",
     "response": "<VIDEO:https://youtu.be/mS41bHNw1WI?si=idnXKPox1kYTav-e>"},
    {"question": "supermax song", "intent": "max",
     "response": "<VIDEO:https://youtu.be/mS41bHNw1WI?si=idnXKPox1kYTav-e>"},
    {"question": "เพลง max", "intent": "max",
     "response": "<VIDEO:https://youtu.be/mS41bHNw1WI?si=idnXKPox1kYTav-e>"}
]

# แปลงเป็น DataFrame แล้วต่อกับของเดิม
df_new = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

# เซฟทับไฟล์
df_new.to_excel("q_a_dataset.xlsx", index=False)

print("✅ เพิ่ม intent 'max' พร้อม response YouTube แล้ว บันทึกเรียบร้อย")


✅ เพิ่ม intent 'max' พร้อม response YouTube แล้ว บันทึกเรียบร้อย


In [117]:
import pandas as pd

df = pd.read_excel("q_a_dataset.xlsx")
print("Columns:", df.columns)
print(df.head())



Columns: Index(['question', 'intent', 'response'], dtype='object')
                                question                intent  \
0                 บอทนี้ทำอะไรได้บ้าง!!!          capabilities   
1             TEAM PRINCIPAL ทีมเรดบูล 𓄀        team_principal   
2    Max vs Lewis อันดับคะแนนเท่าไหร่!!!       team_comparison   
3             ฐานทีม Red Bull อยู่ที่ไหน                 bases   
4  Who is the Technical Director of RBR?  technical_director_e   

                                            response  
0  ฉันตอบคำถามเกี่ยวกับ F1 Team Red Bull 2025: นั...  
1  หัวหน้าทีม (Team Principal/CEO) คือ Laurent Me...  
2  อันดับนักขับ Max Verstappen อยู่อันดับ 3 ด้วยค...  
3  แชสซีส์อยู่ Milton Keynes, UK; เครื่องยนต์อยู่...  
4       The Technical Director is Pierre Waché. 🏁!!!  


In [134]:
# Save model
joblib.dump(model, "chatbot_model.pkl")
print("Model saved as chatbot_model.pkl")

Model saved as chatbot_model.pkl


In [135]:
import random
def chatbot_response(user_input: str):
    intent = model.predict([user_input])[0]
    fallback = f"ขอโทษค่ะ บอทนี้ตอบเฉพาะ {FACTS['team_short']} ฤดูกาล {FACTS['year']} เท่านั้น"
    return random.choice(responses_dict.get(intent, [fallback]))

print(chatbot_response("คะแนนของ Max Verstappen ปี 2025 เท่าไหร่"))

อัปเดตถึง ITALIAN GRAND PRIX (2025-09-10) — MAX VERSTAPPEN อันดับ P3 มี 230 คะแนน; LIAM LAWSON อันดับ P15 มี 20 คะแนน. 🔥??


In [136]:
%%writefile app.py
import os
import streamlit as st
import pandas as pd
import random
import joblib

# ปิด usage stats
os.environ["STREAMLIT_BROWSER_GATHERUSAGESTATS"] = "false"

MODEL_PATH = "/content/chatbot_model.pkl"
DATASET_PATH = "/content/q_a_dataset.xlsx" # แก้ไขชื่อไฟล์ให้ถูกต้อง

# โหลดโมเดล/ชุดคำตอบ
model = joblib.load(MODEL_PATH)
df = pd.read_excel(DATASET_PATH)
intent_to_response = df.groupby("intent")["response"].apply(list).to_dict()

# ข้อมูลสำหรับ Red Bull Racing ปี 2025
FACTS = {
    "year": "2025",
    "team_short": "F1 Team Red Bull",
    "team_full": "Oracle Red Bull Racing",
}

st.set_page_config(page_title="F1 Team Red Bull 2025 Chatbot", page_icon="🏎️")
st.title("F1 Team Red Bull 2025 Chatbot")
st.caption("Ask anything about the Oracle Red Bull Racing 2025 season (TH).")

# session state
if "history" not in st.session_state:
    st.session_state.history = []

# input
user_input = st.text_input("You: ", "", placeholder="Ask about F1 Team Red Bull 2025...")

col1, col2 = st.columns([1,1])
with col1:
    send = st.button("Send", type="primary")
with col2:
    clear = st.button("Clear history")

if clear:
    st.session_state.history = []

if send and user_input.strip():
    predicted_intent = model.predict([user_input])[0]

    fallback_message = f"ขอโทษครับ/ค่ะ บอทนี้ตอบเฉพาะ {FACTS['team_full']} ฤดูกาล {FACTS['year']} เท่านั้น"
    bot_reply = random.choice(
        intent_to_response.get(predicted_intent, [fallback_message])
    )
    st.session_state.history.append({"user": user_input, "bot": bot_reply})

# history
for chat in st.session_state.history:
    st.markdown(f"**You:** {chat['user']}")
    # ถ้าเป็น response แบบ VIDEO
    if str(chat['bot']).startswith("<VIDEO:"):
        url = chat['bot'].replace("<VIDEO:", "").replace(">", "").strip()
        st.video(url)
    else:
        st.markdown(f"**🤖 Bot:** {chat['bot']}")


Writing app.py


In [144]:
!ngrok authtoken 2spnOtxlmFEEvSysAUJbemAPDKR_5K4qumN3T2WDnTap3K42E

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [146]:
!streamlit run app.py --server.port 8501 &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.48.67.190:8501

  Stopping...


In [145]:
# เปิดทางเข้า ngrok
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(8501)
print("🌍 Open your app here:", public_url)

🌍 Open your app here: NgrokTunnel: "https://6f06d8563c57.ngrok-free.app" -> "http://localhost:8501"
